## Examples of pyspark ML

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Missing').getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/09 02:24:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/11/09 02:24:26 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/11/09 02:24:26 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
# Read the dataset 
training = spark.read.csv('try2.csv',header=True,inferSchema=True)

In [4]:
training.show()

+-------+---+----------+------+
|   Name|age|Experience|Salary|
+-------+---+----------+------+
|Harshit|  2|         1|   123|
|asdsads|  3|        11|  1234|
| qwerty|  2|         5|    12|
+-------+---+----------+------+



In [5]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [6]:
training.columns

['Name', 'age', 'Experience', 'Salary']

#### [Age,Experience] -> group together to make --> newFeature ------>>> Independent Feature

In [7]:
# add Together
from pyspark.ml.feature import VectorAssembler
feature_assembler= VectorAssembler(inputCols=["age","Experience"], outputCol="Independent Features")

In [8]:
output= feature_assembler.transform(training)

In [9]:
output.show()

+-------+---+----------+------+--------------------+
|   Name|age|Experience|Salary|Independent Features|
+-------+---+----------+------+--------------------+
|Harshit|  2|         1|   123|           [2.0,1.0]|
|asdsads|  3|        11|  1234|          [3.0,11.0]|
| qwerty|  2|         5|    12|           [2.0,5.0]|
+-------+---+----------+------+--------------------+



In [10]:
finalized_data= output.select("Independent Features","Salary")

In [11]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|           [2.0,1.0]|   123|
|          [3.0,11.0]|  1234|
|           [2.0,5.0]|    12|
+--------------------+------+



In [18]:
from pyspark.ml.regression import LinearRegression
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol="Independent Features", labelCol="Salary")
regressor= regressor.fit(train_data)

22/11/09 02:28:56 WARN Instrumentation: [740497bc] regParam is zero, which might cause numerical instability and overfitting.
22/11/09 02:28:56 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/11/09 02:28:56 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/11/09 02:28:56 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK
22/11/09 02:28:56 WARN Instrumentation: [740497bc] Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.


In [19]:
### Coeffecient
regressor.coefficients

DenseVector([0.0, -27.75])

In [20]:
### Intercepts
regressor.intercept

150.75

In [21]:
# Predictions
pred_results = regressor.evaluate(test_data)

In [22]:
pred_results.predictions.show()

+--------------------+------+----------+
|Independent Features|Salary|prediction|
+--------------------+------+----------+
|          [3.0,11.0]|  1234|    -154.5|
+--------------------+------+----------+



In [23]:
pred_results.meanAbsoluteError, pred_results.meanSquaredError

(1388.5, 1927932.25)